In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

workspace = os.getcwd()
load_data_path = workspace + "/data.npz"
load = np.load(load_data_path)

# load data
train_image = load['trainimg']
train_label = load['trainlabel']
test_image = load['testimg']
test_label = load['testlabel']
number_train = train_image.shape[0]
number_test = test_image.shape[0]
number_label = train_label.shape[1]
print('Number of train set : %s\n'%number_train)
print('Number of test set : %s\n'%number_test)


# hyper parameters
learning_rate = 0.001
training_epochs = 50
batch_size = 100

# dropout 
keep_prob = tf.placeholder(tf.float32)

# input place holders
with tf.name_scope('input'):
    X = tf.placeholder(tf.float32, [None, 128*128*3])
X_img = tf.reshape(X, [-1, 128, 128, 3])   # img 128x128x3

with tf.name_scope('y_'):
    Y = tf.placeholder(tf.float32, [None, 5])

# Layer_1 ImgIn shape=(?, 128, 128, 3)

with tf.name_scope("conv_1"):
    W1 = tf.Variable(tf.random_normal([3, 3, 3, 32], stddev=0.01))
    #    Convolution     -> (?, 128, 128, 32)
    #    Max_Pool     -> (?, 64, 64, 32)
    L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME')
    L1 = tf.nn.relu(L1)
    L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME')
    L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

# Layer_2 ImgIn shape=(?, 64, 64, 32)
with tf.name_scope("conv_2"):
    W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
    #    Conv      ->(?, 64, 64, 64)
    #    Pool      ->(?, 32, 32, 64)
    L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
    L2 = tf.nn.relu(L2)
    L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME')
    L2 = tf.nn.dropout(L2, keep_prob=keep_prob)


# Layer_2 ImgIn shape=(?, 64, 64, 32)
with tf.name_scope("conv_3"):
    W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.01))
    #    Conv      ->(?, 64, 64, 64)
    #    Pool      ->(?, 32, 32, 64)
    L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME')
    L3 = tf.nn.relu(L3)
    L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME')
    L3 = tf.nn.dropout(L3, keep_prob=keep_prob)


# L3 ImgIn shape=(?, 32, 32, 64)
with tf.name_scope("conv_4"):
    W4 = tf.Variable(tf.random_normal([3, 3, 128, 256], stddev=0.01))
    #    Conv      ->(?, 32, 32, 128)
    #    Pool      ->(?, 16, 16, 128)
    #    Reshape   ->(?, 16 * 16 * 128) # Flatten them for FC
    L4 = tf.nn.conv2d(L3, W4, strides=[1, 1, 1, 1], padding='SAME')
    L4 = tf.nn.relu(L4)
    L4 = tf.nn.max_pool(L4, ksize=[1, 2, 2, 1], strides=[
                    1, 2, 2, 1], padding='SAME')
    L4 = tf.nn.dropout(L4, keep_prob=keep_prob)
    L4_flat = tf.reshape(L4, [-1, 256 * 8 * 8])

# L4 FC 4x4x128 inputs -> 625 outputs
with tf.name_scope("fc_1"):
    W5 = tf.get_variable("W5", shape=[256 * 8 * 8, 625],
                     initializer=tf.contrib.layers.xavier_initializer())
    b5 = tf.Variable(tf.random_normal([625]))
    L5 = tf.nn.relu(tf.matmul(L4_flat, W5) + b5)
    L5 = tf.nn.dropout(L5, keep_prob=keep_prob)

# L5 Final FC 625 inputs -> 5 outputs
with tf.name_scope("output_layer"):
    W6 = tf.get_variable("W6", shape=[625, 5],
                     initializer=tf.contrib.layers.xavier_initializer())
    b6 = tf.Variable(tf.random_normal([5]))
    logits = tf.matmul(L5, W6) + b6

# define cost/loss & optimizer
with tf.name_scope("cost"):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
        logits=logits, labels=Y))
    cost_sum = tf.summary.scalar("cost",cost)

with tf.name_scope("Train") as scope:    
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

#accuracy
with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))    
    tf.summary.scalar('accuracy', accuracy)


sess=tf.Session()
# tensorboard --logdir=./logs/xor_logs
merged_summary = tf.summary.merge_all()
writer = tf.summary.FileWriter("./board/cnn")
writer.add_graph(sess.graph)  # Show the graph
    
sess.run(tf.global_variables_initializer())

# train model

for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(number_train / batch_size)

    for i in range(total_batch):
        randidx = np.random.randint(number_train,size=batch_size)
        batch_xs = train_image[randidx,:]
        batch_ys = train_label[randidx,:]
        feed_dict = {X: batch_xs, Y: batch_ys, keep_prob: 0.7}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
    if epoch%5 == 0:
        summary_str = sess.run(merged_summary,feed_dict = {X:batch_xs ,Y:batch_ys,keep_prob:1})
        writer.add_summary(summary_str,epoch)
        writer.flush()
        print('Epoch:', '%03d' % (epoch + 1), 'cost =', '{:.5f}'.format(avg_cost))

print('Finished')


print('Accuracy:', sess.run(accuracy, feed_dict={
      X: test_image, Y: test_label, keep_prob: 1}))

# predict

print("Label: ", sess.run(tf.argmax(test_label[0:1], 1)))
print("Prediction: ", sess.run(
    tf.argmax(logits, 1), feed_dict={X:test_image[0:1], keep_prob: 1}))

plt.imshow(test_image[0:1].
           reshape(128,128,3), cmap='Greys', interpolation='nearest')
plt.show()

Number of train set : 196

Number of test set : 50

('Epoch:', '001', 'cost =', '2.33503')
('Epoch:', '006', 'cost =', '1.96295')
